<h1>Assignment 3: Group Project</h1>

<h3>Movie Rating Prediction system</h3>

Group 6 Members:
- Vo Thanh Luan (s3822042)
- Nguyen Bao Khang (s3817970)
- Duong Tuan Dat (s3636739)
- Bui Dang Dac Duong (s3764487)

In [ ]:
# import libraries

import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import re

import warnings 
warnings.filterwarnings('ignore')

# 1. Data preparation and exploration
---

## 1.1 Data Retrieving

In [ ]:
movies_df = pd.read_csv('data/movies.csv', delimiter=',', parse_dates=["release_date"])
ratings_df = pd.read_csv('data/ratings.csv', delimiter=',')

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

## 1.2 Data cleaning and preparation

In [ ]:
#set threshold
threshold = 0.65

# dropping columns with missing value rate higher than threshold
ratings_df = ratings_df[ratings_df.columns[ratings_df.isnull().mean() < threshold]]

# Remove the missing values from the rows having greater or equal to 6 missing values.
movies_df = movies_df[movies_df.isnull().sum(axis=1) <= 6]
ratings_df = ratings_df[ratings_df.isnull().sum(axis=1) <= 6]

### 1.2.1 Handle missing data

In [ ]:
# Percentage of missing value in movies_df before cleaning
percent_missing = movies_df.isnull().sum() * 100 / len(movies_df)
missing_values = pd.DataFrame({'percent_missing': percent_missing})
missing_values.sort_values(by ='percent_missing' , ascending=False)

In [ ]:
# Filling missing value for movies dataset

movies_df["tagline"] = movies_df["tagline"].fillna("Unknown")

movies_df["overview"] = movies_df["overview"].fillna("Unknown")

movies_df["poster_path"] = movies_df["poster_path"].fillna("Unknown")

movies_df["runtime"] = movies_df["runtime"].fillna(method='ffill')

movies_df["release_date"] = movies_df["release_date"].fillna(method='ffill')

movies_df["status"] = movies_df["status"].fillna(movies_df['status'].mode()[0])

movies_df["imdb_id"] = movies_df["imdb_id"].fillna(method='ffill')

movies_df["original_language"] = movies_df["original_language"].fillna(movies_df['original_language'].mode()[0])

In [ ]:
# Percentage of missing value in movies_df after cleaning
percent_missing = movies_df.isnull().sum() * 100 / len(movies_df)
missing_values = pd.DataFrame({'percent_missing': percent_missing})
missing_values.sort_values(by ='percent_missing' , ascending=False)

In [ ]:
# Percentage of missing value in ratings_df before cleaning
percent_missing = ratings_df.isnull().sum() * 100 / len(ratings_df)
missing_values = pd.DataFrame({'percent_missing': percent_missing})
missing_values.sort_values(by ='percent_missing' , ascending=False)

In [ ]:
# rename column id in movies_df to match movieId with ratings_df
movies_df.rename(columns={"id": "movieId"}, inplace=True)

In [ ]:
# Check type for movies_df
movies_df.dtypes

In [ ]:
# Check type for ratings_df
ratings_df.dtypes

In [ ]:
# Delete row in movieId column contain string value
movies_df = movies_df[~movies_df['movieId'].isin(['1997-08-20', '2012-09-29','2014-01-01'])]

In [ ]:
# change column type for movies_df
movies_df["movieId"] = pd.to_numeric(movies_df['movieId'])

movies_df["popularity"] = pd.to_numeric(movies_df['popularity'])

movies_df["budget"] = pd.to_numeric(movies_df['budget'])

Each movie has many ratings rated by many different users. We calculate the average ratings of one movie by sum all the ratings of one movie and divide by the number of ratings.

In [ ]:
ratings_df = ratings_df.groupby(['movieId']).mean().groupby('movieId')['rating'].mean().reset_index()

In [ ]:
df = movies_df.merge(ratings_df, how="left", on="movieId")

In [ ]:
df['rating'].isnull().sum()

As the number of movies which do not have rating is very high(42621); therefore, we need to handle missing data. In order to do that, we see that in the data frame it has the "imdb_id" column. Therefore,we will use that column to crawl the updated data of movie rating in IMDB website. When doing this, we can make sure the movie rating is accurate and updated as well as taking care of missing rating in movies.

Refer to "imdb_crawler" folder , we have created the python script to crawl rating from IMDB website and then we output the updated ratings.csv file.

In [ ]:
# Load new ratings.csv file
ratings_updated_df = pd.read_csv("imdb_crawler/ratings.csv", delimiter=",")

In [ ]:
# Percentage of missing value in ratings_updated_df before cleaning
percent_missing = ratings_updated_df.isnull().sum() * 100 / len(ratings_updated_df)
missing_values = pd.DataFrame({'percent_missing': percent_missing})
missing_values.sort_values(by ='percent_missing' , ascending=False)

As the percentage of missing values in rating column is very low (0.05%), we can safely drop the rows with missing values.

In [ ]:
ratings_updated_df['rating'] = ratings_updated_df['rating'].dropna(how='any',axis=0)

In [ ]:
ratings_updated_df = ratings_updated_df.groupby(['movieId']).mean().groupby('movieId')['rating'].mean().reset_index()

In [ ]:
df_updated = movies_df.merge(ratings_updated_df, how="left", on="movieId")

Round rating float, for classification

In [ ]:
df_updated = df_updated.round({'rating': 0})

Check if any movies do not have rating

In [ ]:
df_updated['rating'].isnull().sum()

As the number that do not have rating is very small (49) compared to the ammount of movies, therefore, we will delete all the rows contains the movie which do not have rating.

In [ ]:
df_updated = df_updated.dropna(subset=['rating'])

In this step, we check the number of movies which do not have rating and see that all the movies have its rating now.

In [ ]:
df_updated['rating'].isnull().sum()

In the dataset, there are some columns with array object as values. We would like to convert to array string based on the field name

In [ ]:
from ast import literal_eval

def split_genres(row):
    row['genres'] = ",".join([info['name'] for info in literal_eval(row['genres'])])
    return row

def split_prod_countries(row):
    row['production_countries'] = ",".join([info['name'] for info in literal_eval(row['production_countries'])])
    return row

def split_prod_companies(row):
    row['production_companies'] = ",".join([info['name'] for info in literal_eval(row['production_companies'])])
    return row

def split_spoken_languages(row):
    row['spoken_languages'] = ",".join([info['name'] for info in literal_eval(row['spoken_languages'])])
    return row
  
df_updated = df_updated.apply(split_genres, axis=1)
df_updated = df_updated.apply(split_prod_countries, axis=1)
df_updated = df_updated.apply(split_prod_companies, axis=1)
df_updated = df_updated.apply(split_spoken_languages, axis=1)

Function for text cleaning

In [ ]:
def clean_text(text):
    # remove backslash-apostrophe
    text = re.sub("\'", "", text)
    # remove everything alphabets
    text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    # remove whitespaces
    text = ' '.join(text.split())
    # remove double quote inside
    text = text.replace('“', '').replace('”', '').replace('"', '')
    # remove number
    text = re.sub(r"\d+", "", text)
    # convert text to lowercase
    text = text.lower()
    
    return text

Apply clean_text function for below column

In [ ]:
df_updated['title'] = df_updated['title'].apply(lambda x: clean_text(x))
df_updated['tagline'] = df_updated['tagline'].apply(lambda x: clean_text(x))
df_updated['original_title'] = df_updated['original_title'].apply(lambda x: clean_text(x))
df_updated['overview'] = df_updated['overview'].apply(lambda x: clean_text(x))

Remove stopword

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)
  
df_updated['overview'] = df_updated['overview'].apply(lambda x: remove_stopwords(x))

Create original data frame by copying exising dataframe before doing categorical encoding

In [ ]:
df_raw = df_updated.copy()

**belongs_to_collection**


Convert column 'belongs_to_collection' data to binary data of 0 and 1 (1 means the movie belongs to a collection, and 0 means it does not belong to any collection).

In [ ]:
# converting column 'belongs_to_collection' to binary 0 and 1 data
df_updated["belongs_to_collection"] = df_updated["belongs_to_collection"].fillna(0)
df_updated["belongs_to_collection"] = np.where(df_updated["belongs_to_collection"] != 0,1,0)
df_updated['belongs_to_collection'] = pd.to_numeric(df_updated['belongs_to_collection'])

**homepage**

Convert column 'homepage' to binary data of 0 and 1 (1 means the movie has a homepage, and 0 means the movie does not have a homepage)

In [ ]:
df_updated["homepage"] = df_updated["homepage"].fillna(0)
df_updated["homepage"] = np.where(df_updated["homepage"] != 0,1,0)
df_updated['homepage'] = pd.to_numeric(df_updated['homepage'])

**adult**

Convert column 'adult' data from boolean value to 1 and 0 (1: True, 0: False)

In [ ]:
df_updated["adult"] = np.where(df_updated['adult'] == True, 1, 0)
df_updated['adult'] = pd.to_numeric(df_updated['adult'])

**video**

Convert column 'video' data from boolean value to 1 and 0 (1: True, 0: False)

In [ ]:
df_updated["video"] = np.where(df_updated['video'] == True, 1, 0)
df_updated['video'] = pd.to_numeric(df_updated['video'])

**status**

In [ ]:
df_updated['status'].value_counts()

Since "Released" is the dominant value in this column, we can make the dataset easier for computing by replacing value "Released" with 1, and the others with 0.

In [ ]:
df_updated["status"] = np.where(df_updated['status'].str.contains("Released"), 1, 0)
df_updated['status'] = pd.to_numeric(df_updated['status'])

**spoken_languages**

In [ ]:
df_updated['spoken_languages'].value_counts()

Same with column 'status', English is the most common language used in the movies. In addition, it also dominate other languages in terms of quantity. Therefore, we can set value 1 for movies that has English as one of the spoken language, and 0 when the movie does not use English.

In [ ]:
df_updated["spoken_languages"] = np.where(df_updated['spoken_languages'].str.contains("English"), 1, 0)
df_updated['spoken_languages'] = pd.to_numeric(df_updated['spoken_languages'])

**original_language**

In [ ]:
df_updated['original_language'].value_counts()

In [ ]:
df_updated["original_language"] = np.where(df_updated['original_language'] == "en", 1, 0)
df_updated['original_language'] = pd.to_numeric(df_updated['original_language'])

**production_countries**

In [ ]:
df_updated['production_countries'].value_counts()

From a real world perspective, we can observe that the US's movie industry is gigantic. From this standpoint, we can set value of 1 for movies that involved the US in the production phase, and 0 for ther others.

In [ ]:
df_updated["production_countries"] = np.where(df_updated['production_countries'].str.contains("United States of America"), 1, 0)
df_updated['production_countries'] = pd.to_numeric(df_updated['production_countries'])

**genres**

In [ ]:
genre_list = []
for value in df_updated['genres'].unique():
  value_list = value.split(',')
  for item in value_list:
    genre_list.append(item.strip())

genre_list = np.unique(np.array(genre_list)).tolist()
genre_list.remove("")

for genre in genre_list:
  column_name = 'genres_' + genre
  df_updated[column_name] = np.where(df_updated['genres'].str.contains(genre), 1, 0)
    
df_updated.drop("genres",axis=1,inplace=True)

**production_companies**

In [ ]:
df_updated['production_companies'].value_counts()

Out of over 45k movies, there are x movies that does not have any production company. In this situation, we can perform custom binary encoding for this feature, so that value 1 represents movies that has at least one production company, and reverse, value 0 represents movies that does not have any production company


In [ ]:
df_updated["production_companies"] = np.where(df_updated['production_companies'] != "", 1, 0)

In [ ]:
df_updated['production_companies'].value_counts()

**release_date**

To use a datetime feature for fitting into a machine learning model, we can extract cyclical features (day, month) and encode them using sine and cosine transformation. This is to ensure that the data still keeps there cyclinal nature.
https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning

In [ ]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [ ]:
df_updated['release_date']= pd.to_datetime(df_updated['release_date'])
df_updated['month'] = df_updated.release_date.dt.month
df_updated['day'] = df_updated.release_date.dt.dayofyear

In [ ]:
df_updated = encode(df_updated, 'month', 12)
df_updated = encode(df_updated, 'day', 365)
# drop original unencoded features
df_updated.drop(['month', 'day', 'release_date'],axis=1,inplace=True)

## 1.3 Data Exploration

Graph 1: Number of Movies produced by a Country

In [ ]:
from collections import Counter

a = []
for each in df_raw.production_countries.str.split(","):
    for i in each:
        a.append(i)
        
b = dict(Counter(a))

keys = []
values = []

for key,value in b.items() :
    if value > 500 and key != "":
        keys.append(key)
        values.append(value)

       
labels = keys
colors = sns.color_palette()
explode = [0,0,0,0,0,0,0,0,0,0,0,0,0]
sizes = values
    
plt.figure(figsize = (20,12))

plt.pie(sizes,explode = explode,labels = labels,colors = colors,autopct = '%1.1f%%',textprops = {"fontsize": 20},shadow = False)
plt.title("Number of Movies produced by a Country", fontsize = 20)

plt.show()

Graph 2: Number of Each Movie Genres 

In [ ]:
from collections import Counter

data = df_raw.genres.copy()

list_kind = data.str.split(",")
a = []
for each in list_kind:
    for i in each:
        a.append(i)
        
c=[]
for each in a:
    if each != "":
         c.append(each)        
        
f= dict(Counter(c))

df3 = pd.DataFrame(list(f.items()),columns = ["kind","ratio"])
new_index =( df3.ratio.sort_values(ascending=False)).index.values
new = df3.reindex(new_index)

plt.figure( figsize = (20,12))
plt.tick_params(labelsize = 15)
sns.barplot(x="kind",y="ratio",data=new,palette = sns.cubehelix_palette(len(f)))
plt.xticks(rotation = 90)
plt.xlabel("Kind Of Movie",fontsize=20)
plt.ylabel("Count",fontsize=20)
plt.title("Number of Each Movie Genres",fontsize = 20)

Graph 3: Count Each Type of Movie Ratings 

In [ ]:
f,ax = plt.subplots(1,1,figsize=(20,12))
ax = sns.countplot(x="rating", data=ratings_updated_df)
ax.set_yticklabels([num for num in ax.get_yticks()])
plt.tick_params(labelsize = 15)
plt.title("Count Each Type of Movie Ratings ", fontsize = 20)
plt.xlabel("Ratings", fontsize = 20)
plt.ylabel("Number of Ratings", fontsize = 20)
plt.show()

Graph 4: Most Popular Movies

In [ ]:
pop = df_raw.sort_values('popularity', ascending=False)
plt.figure(figsize=(20,12))

plt.barh(pop['title'].head(6),pop['popularity'].head(6), align='center',
        color='skyblue')
plt.gca().invert_yaxis()
plt.tick_params(labelsize = 15)
plt.xlabel("Popularity", fontsize = 20)
plt.ylabel("The title of movie", fontsize = 20)
plt.title("Most Popular Movies", fontsize = 20)

Graph 5: Top 6 movies with highest revenue compared to its budget

In [ ]:
df1 = movies_df[['title','revenue','budget']].copy()

df1.set_index('title', inplace=True)

ax = df1.sort_values(by=['revenue'], ascending=False).head(6).plot(kind='bar', figsize=(20,12))
ax.xaxis.set_major_formatter(plt.FixedFormatter(df1.index.to_series()))
plt.tick_params(labelsize = 15)
plt.ylabel("Ammount of money in billion ($)", fontsize = 20)
plt.xlabel("The title of movie", fontsize = 20)
plt.legend(fontsize=20)
plt.title("Top 6 movies with highest revenue compared to its budget", fontsize = 20)
plt.show()

Chart 6: Most popular Language in Movies

In [ ]:
unique = list(df_raw.original_language.unique())
list_ratio=[]
for each in unique:
    x= movies_df[movies_df["original_language"] == each]
    ratio_popularity=sum(x.popularity)/len(x)
    list_ratio.append(ratio_popularity)
    
df1 = pd.DataFrame({"language":unique,"ratio":list_ratio})
new_index = (df1.ratio.sort_values(ascending = False).head(6)).index.values
sorted_data= df1.reindex(new_index)

#Visualization
plt.figure(figsize = (20,12))
sns.barplot(x= sorted_data["language"],y  = sorted_data["ratio"])
plt.xticks(rotation= 90)
plt.tick_params(labelsize = 15)

plt.xlabel("Language",fontsize = 20)
plt.ylabel("Popularity",fontsize = 20)
plt.title("Most popular Language in Movies", fontsize = 20)

# 2. Data Modelling
---

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
import xgboost as xgb

## 2.1 Feature Engineering

Feature selection for fitting into the machine learning algorithms

In [ ]:
df_updated.head()

#### 2.1.1 Reasoning

In [ ]:
df_updated['adult'].value_counts()

Because column 'adult' only contains 1 value (False), we can remove this feature from the dataset. Also, from a reality perspective, we can also drop other columns that does not predict the rating of a movies:
- imdb_id: id of the movie on imdb film review site
- movieId: id of the movie in the dataset
- poster_path: url to the poster of the movie
- original_title, title: name of the movie before and after release
- overview: a brief description about the movie
- tagline: the movie quote (usually in posters)

In [ ]:
df_updated.drop(['adult','imdb_id','poster_path','title','original_title','overview','movieId','tagline'], axis=1, inplace=True)

In [ ]:
first_column = df_updated.pop('rating')
df_updated.insert(0, 'rating', first_column)

In [ ]:
df_updated['rating'].value_counts()

#### 2.1.2 Correlation Matrix

In [ ]:
f,ax = plt.subplots(figsize = (20,12))
sns.heatmap (df_updated.corr(), annot = True,linewidths =0.75,linecolor = "White",fmt = ".2f",ax = ax,center = -0.1)
plt.show()

### 2.2 Train/Test/Val Split

In [ ]:
train_size = 0.8
valid_size = 0.1

train_index = int(len(df_updated)*train_size)

df_updated_train = df_updated[0:train_index]
df_updated_rem = df_updated[train_index:]

valid_index = int(len(df_updated)*valid_size)

df_updated_valid = df_updated[train_index:train_index+valid_index]
df_updated_test = df_updated[train_index+valid_index:]

X_train, y_train = df_updated_train.drop(columns='rating').copy(), df_updated_train['rating'].copy()
X_valid, y_valid = df_updated_valid.drop(columns='rating').copy(), df_updated_valid['rating'].copy()
X_test, y_test = df_updated_test.drop(columns='rating').copy(), df_updated_test['rating'].copy()

### 2.3 Parameter tuning

In [ ]:
from skopt import BayesSearchCV
from sklearn.model_selection import RandomizedSearchCV

Parameter Tunning for Decision Tree Algorithm

In [ ]:
n_iter = 20

paramDT = {
    'max_depth' : [3,5,7,9,10,15,20,25],
    'criterion' : ['gini', 'entropy'],
    'max_features' : ['auto', 'sqrt', 'log2'],
    'min_samples_split' : [2,4,6]
}

Parameter Tunning for Light GBM Algorithm

In [ ]:
paramLGBM = {
    'num_leaves':[20,40,60,80,100], 
    'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],
    'learning_rate':[0.05,0.1,0.2],
    'reg_alpha':[0,0.01,0.03]
}

Parameter Tunning for XG Boost Algorithm

In [ ]:
paramXGBR = { 
    'max_depth': [3, 5, 6, 10, 15, 20],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': np.arange(0.5, 1.0, 0.1),
    'colsample_bytree': np.arange(0.4, 1.0, 0.1),
    'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
    'n_estimators': [100, 500, 1000]
}

### 2.4 Model training

Model 1: Using Decision Tree Algorithm

In [ ]:
# DT_searchCV  = BayesSearchCV(
#         estimator = DecisionTreeClassifier(random_state=42),
#         search_spaces = paramDT,
#         n_jobs = -1,
#         cv = 3,
#         n_iter  = n_iter,
#         scoring = "accuracy",
#         verbose = 4,
#         random_state = 42
# )
# DT_searchCV.fit(X_train,y_train)

In [ ]:
# DT_searchCV.best_estimator_

Best params for this algorithm:
{
    'max_depth': 15,
    'max_features': 'auto',
    'random_state': 42
}

In [ ]:
dt = DecisionTreeClassifier(max_depth=15, max_features='auto', random_state=42)
dt.fit(X_train, y_train)
y_predict_dt = dt.predict(X_test)

Model 2: Using LightGBM

In [ ]:
lgbm = lgb.LGBMClassifier(random_state=42)
# LGBM_searchCV = RandomizedSearchCV(
#     estimator = lgb,
#     param_distributions = paramLGBM,
#     n_jobs = -1,
#     scoring = 'accuracy',
#     n_iter = 20,
#     random_state=42
# )
# LGBM_searchCV.fit(X_train, y_train)

In [ ]:
# LGBM_searchCV.best_params_

Best Parameters for this algorithm:
{'reg_alpha': 0.03,
 'num_leaves': 20,
 'min_child_samples': 15,
 'max_depth': 5,
 'learning_rate': 0.05}

In [ ]:
lgbm_params = {
'reg_alpha': 0.03,
 'num_leaves': 20,
 'min_child_samples': 15,
 'max_depth': 5,
 'learning_rate': 0.05,
 'random_state':42
 }

In [ ]:
#set optimal parameters
lgbm.set_params(**lgbm_params)
lgbm.fit(X_train, y_train)
y_predict_lgbm = lgbm.predict(X_test)

Model 3: Using XGBoost Algorithm

In [ ]:
xgbr = xgb.XGBClassifier(random_state=42)
# XGBR_searchCV = RandomizedSearchCV(
#     estimator = xgbr,
#     param_distributions = paramXGBR,
#     scoring = 'accuracy',
#     n_iter = 10,
#     n_jobs = -1,
#     random_state=42
# )

# XGBR_searchCV.fit(X_train, y_train)

In [ ]:
# XGBR_searchCV.best_params_

Best Parameters for this algorithm:
{
    'subsample': 0.8999999999999999,
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'colsample_bytree': 0.5,
    'colsample_bylevel': 0.8999999999999999
 }

In [ ]:
xgbr_params = {
    'subsample': 0.8999999999999999,
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'colsample_bytree': 0.5,
    'colsample_bylevel': 0.8999999999999999,
    'random_state':42
 }

In [ ]:
xgbr.set_params(**xgbr_params)
xgbr.fit(X_train, y_train)
y_predict_xgbr = xgbr.predict(X_test)

### 2.5 Model Evaluation and Selection

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

Model 1: Using Decision Tree Algorithm

In [ ]:
dt_rmse = np.sqrt(mean_squared_error(y_test, y_predict_dt))
dt_mae = mean_absolute_error(y_test, y_predict_dt)
dt_r2 = r2_score(y_test, y_predict_dt)
dt_accuracy = accuracy_score(y_test, y_predict_dt)

print('RMSE: ', dt_rmse)
print('MAE: ', dt_mae)
print('R2: ', dt_r2)
print('Accuracy: ', dt_accuracy)

Model 2: Using LightGBM Algorithm

In [ ]:
lgbm_rmse = np.sqrt(mean_squared_error(y_test, y_predict_lgbm))
lgbm_mae = mean_absolute_error(y_test, y_predict_lgbm)
lgbm_r2 = r2_score(y_test, y_predict_lgbm)
lgbm_accuracy = accuracy_score(y_test, y_predict_lgbm)

print('RMSE: ', lgbm_rmse)
print('MAE: ', lgbm_mae)
print('R2: ', lgbm_r2)
print('Accuracy: ', lgbm_accuracy)

Model 3: Using XGBoost Algorithm

In [ ]:
xgbr_rmse = np.sqrt(mean_squared_error(y_test, y_predict_xgbr))
xgbr_mae = mean_absolute_error(y_test, y_predict_xgbr)
xgbr_r2 = r2_score(y_test, y_predict_xgbr)
xgbr_accuracy = accuracy_score(y_test, y_predict_xgbr)

print('RMSE: ', xgbr_rmse)
print('MAE: ', xgbr_mae)
print('R2: ', xgbr_r2)
print('Accuracy: ', xgbr_accuracy)

In [ ]:
model_metric = {
    'Model': ['Decision Tree Algorithm', 'Light GBM Algorithm', 'XG Boost ALgorithm'],
    'RMSE': [dt_rmse,lgbm_rmse , xgbr_rmse],
    'MAE': [dt_mae, lgbm_mae, xgbr_mae],
    'R2': [dt_r2, lgbm_r2, xgbr_r2],
    'Accuracy': [dt_accuracy,lgbm_accuracy,xgbr_accuracy]
}
model_df = pd.DataFrame(data = model_metric)
model_df.style.background_gradient().hide_index()

### 2.6 Save the model for deployment

In this task, we use Pickle to save and load the model. Since in order to use the model outside of the notebook, we need to sae it - and then later, we can load and use it for deployment 

In [ ]:
import pickle

pickle.dump(xgbr, open('xgbr.pkl', 'wb'))

## 3. Model deployment and Automation
---

Please use Postman in order to do below step:
 - Create new POST request to http://127.0.0.1:5000/evaluate or http://127.0.0.1:5000/predict
  - Body:
    - formdata:
      - key : file
      - type: file
      - value: choose file data.csv in "test" folder
 - Evaluate API: http://127.0.0.1:5000/evaluate to output the evaluation metric
 - PRedict API http://127.0.0.1:5000/predict to output the predicted rating value

## 4. Visualisation Dashboard
---

For starting the visualization dashboard:
 - Go inside folder "movie-rating-prediction'
 - Run python dashboard.py
 - Wait until  this line "Running on http://127.0.0.1:5000/" appear
 - Navigate to this URL to see the visualization dashboard